In [1]:
import pandas as pd
import sys
import os
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install xlrd

    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^
    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^


In [24]:
# Step 1: Load the full Kelly Project Italian dataset
# Download from: https://ssharoff.github.io/kelly/
# Use the Italian file: "itfull.csv" or "kelly_italian.csv"
df = pd.read_excel("it_m3.xls")

# Step 2: Inspect available columns (uncomment to preview)
print(df.columns)
#print(df.head(10))

# Step 3: Define mapping from Points to CEFR levels
points_to_cefr = {
    1: "A1",
    2: "A2",
    3: "B1",
    4: "B2",
    5: "C1",
    6: "C2"
}

# Step 4: Apply CEFR mapping
#df["cefr_level"] = df["Points"].map(points_to_cefr)
df["cefr_level"] = df["Points"]

# Step 5: Filter relevant columns
df_cefr = df[["Lemma", "cefr_level"]].rename(columns={"Lemma": "word"})

# Step 6: Drop rows with missing CEFR level
df_cefr = df_cefr.dropna(subset=["cefr_level"])
print(df_cefr.tail(10))

# Step 7: Save to CSV
df_cefr.to_csv("italian_cefr_from_kelly_full.csv", index=False)

print("✅ Saved full CEFR vocabulary to 'italian_cefr_from_kelly_full.csv'")


    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^
    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^
Index(['Lemma', 'Pos', 'Points'], dtype='object')
              word cefr_level
5339       liquido         C2
5340   raggruppare         C2
5341        ridare         C2
5342    ospitalità         C2
5343      pomodoro         C2
5344     pescatore         C2
5345    chirurgico         C2
5346  implementare         C2
5347         umore         C2
5348    subentrare         C2
✅ Saved full CEFR vocabulary to 'italian_cefr_from_kelly_full.csv'


In [25]:
# Load CEFR-tagged word list (output from Kelly Project)
df_cefr = pd.read_csv("italian_cefr_from_kelly_full.csv")

# Load HermitDave frequency list (format: word frequency)
df_freq = pd.read_csv("italian_freq.txt", sep=" ", names=["word", "frequency"])

# Optional: normalize word column
df_cefr["word"] = df_cefr["word"].str.lower()
df_freq["word"] = df_freq["word"].str.lower()

# Merge on 'word'
merged_df = pd.merge(df_cefr, df_freq, on="word", how="left")

# Optional: Drop words without frequency info (or fill with default)
merged_df = merged_df.dropna(subset=["frequency"])

# Sort by CEFR level and frequency (higher frequency = lower rank number)
merged_df = merged_df.sort_values(by=["cefr_level", "frequency"], ascending=[True, False])

# Save final output
merged_df.to_csv("italian_cefr_vocab_with_freq.csv", index=False)

print("✅ Merged CEFR + frequency list saved to 'italian_cefr_vocab_with_freq.csv'")


✅ Merged CEFR + frequency list saved to 'italian_cefr_vocab_with_freq.csv'


In [4]:
def load_vocab_data(path="italian_cefr_vocab_with_freq.csv"):
    df = pd.read_csv(path)
    df["word"] = df["lemma"].str.lower()
    return df

def recommend_words(user_known_words, user_cefr_level, vocab_df, top_n=20):
    # Define CEFR progression
    cefr_order = ["A1", "A2", "B1", "B2", "C1", "C2"]
    level_index = cefr_order.index(user_cefr_level)
    target_levels = cefr_order[max(0, level_index):min(len(cefr_order), level_index + 2)]

    # Filter by CEFR level and remove known words
    filtered = vocab_df[
        (vocab_df["cefr_level"].isin(target_levels)) &
        (~vocab_df["word"].isin(user_known_words))
    ]

    # Sort by frequency descending (most common first)
    filtered = filtered.sort_values(by="frequency", ascending=False)

    return filtered.head(top_n)[["word", "cefr_level", "frequency"]]


In [27]:
# Load vocab data
vocab_df = load_vocab_data()

# Example user profile
user_known_words = {"essere", "avere", "fare", "dire", "andare", "parlare"}
user_cefr_level = "B1"

# Get recommendations
recommendations = recommend_words(user_known_words, user_cefr_level, vocab_df, top_n=10)
print(recommendations)


        word cefr_level  frequency
1984    cosa         B1  1462858.0
2947      no         B2  1433410.0
1985     mio         B1   752241.0
2948   tutto         B2   704866.0
1986     era         B1   657610.0
2949   detto         B2   530015.0
1987   molto         B1   430250.0
2950  perché         B2   417508.0
2951   prima         B2   387026.0
1988     suo         B1   381756.0


In [13]:
# Load your main vocabulary file (without POS or outdated POS)
main_df = pd.read_csv("italian_cefr_vocab_with_freq.csv")
main_df["word"] = main_df["word"].str.lower()

# Load the source Excel file that contains the correct POS info
# Adjust the sheet name or columns if needed
source_df = pd.read_excel("it_m3.xls")
source_df["word"] = source_df["Lemma"].str.lower()

# Inspect columns to identify the correct POS column
print(source_df.columns)

# Merge on 'word' column, keeping POS from source
merged_df = pd.merge(main_df, source_df[["word", "Pos"]], on="word", how="left")

# Save the updated file
merged_df.to_csv("italian_cefr_vocab_with_freq_pos.csv", index=False)

print("✅ Updated file saved as 'italian_cefr_vocab_with_freq_pos.csv'")

Index(['Lemma', 'Pos', 'Points', 'word'], dtype='object')
✅ Updated file saved as 'italian_cefr_vocab_with_freq_pos.csv'
